We will cover the following recipes:
* Creating datasets
* Creating a YOLO-v3 model
* Defining the loss function
* Training the model
* Deploying the model

## Creating datasets

### Getting ready
1. Download the following GitHub repository: https://github.com/pjreddie/darknet
2. From the download repository, get the darknet/scripts/get_coco_dataset.sh file.
3. Create a folder named data where your scripts are located and copy get_coco_dataset.sh into the folder.
4. Next, open a Terminal and execute the get_coco_dataset.sh file from the data folder. The script will download the complete COCO dataset into a subfolder named coco.
5. Create a folder named config where your scripts are located and copy the darknet/cfg/yolov3.cfg file into the config folder.

## Creating a custom COCO dataset

### 1. First, we will define a custom dataset class for COCO.

In [1]:
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms.functional as TF
import os
import numpy as np
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
# Define the CocoDataset class:
class CocoDataset(Dataset):
    def __init__(self, path2_list_file, transform=None, trans_params=None):
        with open(path2_list_file, "r") as file:
            self.path2imgs = file.readlines()
        self.path2labels = [path.replace("images", "labels").replace(".png", ".txt").replace(".jpg", ".txt")
                            for path in self.path2imgs]
        print(self.path2labels)
        
        self.trans_params = trans_params
        self.transform = transform
        
    def __len__(self):
        return len(self.path2imgs)
    
    def __getitem__(self, index):
        path2img = self.path2imgs[index % len(self.path2imgs)].rstrip()
        
        labels = None
        if os.path.exists(self.path2labels):
            labels = np.loadtxt(self.path2labels).reshape(-1, 5)
        if self.transform:
            img, labels = self.transform(img, labels, self.trans_params)
            
        return img, labels, path2img

### 2. Next, we will create an object of the CocoDataset class for the training data:

In [18]:
root_data = "./data/chapter_5/coco"
path2_train_list = os.path.join(root_data, "trainvalno5k.txt")
coco_train = CocoDataset(path2_train_list)
print(len(coco_train))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
# Get a sample item from coco_train:
img, labels, path2img = coco_train[1]
print("image size:", img.size, type(img))
print("labels shape:", labels.shape, type(labels))
print("labels \n", labels)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not list